In [ ]:
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

library(tidyverse) # metapackage with lots of helpful functions
library(readr)
library(dplyr)
library(tidyr)
library(arules)
library(arulesViz)
library(methods)
library(ggplot2)
## Running code

# In a notebook, you can run a single code cell by clicking in the cell and then hitting 
# the blue arrow to the left, or by clicking in the cell and pressing Shift+Enter. In a script, 
# you can run code by highlighting the code you want to run and then clicking the blue arrow
# at the bottom of this window.

## Reading in files

# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input")

## Saving data

# If you save any files or images, these will be put in the "output" directory. You 
# can see the output directory by committing and running your kernel (using the 
# Commit & Run button) and then checking out the compiled version of your kernel.

**Reading Orders, Products, Departments**

In [ ]:
opp <- read_csv("http://www.utdallas.edu/~ond170030/data/IMBA/order_products__prior.csv")
products <- read_csv("http://www.utdallas.edu/~ond170030/data/IMBA/products.csv")
departments <- read_csv("http://www.utdallas.edu/~ond170030/data/IMBA/departments.csv")

**Now if you see the parsed column names, we can make out what merging has to be done to get the product names, department names against the order_ids**

Merging Prior Orders with products on product_id to get product names and department_id

In [ ]:
ordersWithDepartmentId <-opp %>%
  group_by(product_id) %>%
  left_join(products,by="product_id")


Forming a basket of order_id againts the list of products_names in that particular order

In [ ]:
orderProductBasket <- opp %>% 
  inner_join(products, by="product_id") %>% 
  group_by(order_id) %>%
  summarise(basket = as.vector(list(product_name)))

In [ ]:
Forming a basket of order_id against the list of department in that particular order

In [ ]:
orderDepartmentBasket <- ordersWithDepartmentId %>% 
  inner_join(departments, by="department_id") %>% 
  group_by(order_id) %>%
  summarise(basket = as.vector(list(department)))

Now we are reading (or converting them from) the baskets as transactions

In [ ]:
orderProductTransactions <- as(orderProductBasket$basket, "transactions")
orderDepartmentTransactions <- as(orderDepartmentBasket$basket, "transactions")

**Getting the Frequent Item sets for products**

In [ ]:
# Question 1 
frequentItems <- eclat(orderProductTransactions, parameter = list(supp = 0.01, maxlen=80, minlen=1))
inspect(frequentItems)
itemFrequencyPlot(orderProductTransactions, topN=10, type="absolute", main="Product Frequency")

**Getting Association Rules for products**

In [ ]:
# Question 2
orderProductRules <- apriori (orderProductTransactions, parameter = list(supp=0.01, conf=0.5))
orderProductRulesConf <- sort(orderProductRules, by="confidence", decreasing=TRUE)
inspect(head(orderProductRulesConf))

**Getting the frequent item sets for departments**

In [ ]:
# Question 3
frequentItems <- eclat(orderDepartmentTransactions,parameter = list(supp = 0.1, maxlen=80))
inspect(frequentItems)
itemFrequencyPlot(orderDepartmentTransactions, topN=10, type="absolute", main="Department Frquency")

**Getting Association Rules for departments**

In [ ]:
# Question 4
orderDepartmentRules <-apriori(orderDepartmentTransactions, parameter = list(supp=0.01, conf=0.5))
orderDepartmentRulesConf <- sort(orderDepartmentRules, by="confidence", decreasing=TRUE)
inspect(head(orderDepartmentRulesConf))